In [ ]:
%pip install vit-keras
%pip install tensorflow-addons

In [ ]:
import tensorflow as tf
import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import cv2
from sklearn import preprocessing
from pathlib import Path

In [5]:
labels_train = []
labels_test = []

path_train = []
path_test = []

In [6]:
train_path = r"path-to-train-images"
test_path = r"path-to-test-images"

In [ ]:
import cv2
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

# Function to parse the .xml file and get bounding box details
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    boxes = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        boxes.append((label, xmin, ymin, xmax, ymax))
    return boxes

# Function to visualize the image with bounding boxes and labels
def visualize_image_with_boxes(image_path, boxes):
    image = cv2.imread(image_path)

    for box in boxes:
        label, xmin, ymin, xmax, ymax = box
        # Draw rectangle around the object
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        # Put the label near the object
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Convert BGR image to RGB for displaying with matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image using matplotlib
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

#Usage
xml_file = 'path here '  # Replace with your XML file path
image_file = 'path here'  # Replace with your image path

# Parse XML and visualize
boxes = parse_xml(xml_file)
visualize_image_with_boxes(image_file, boxes)


In [ ]:
import xml.etree.ElementTree as ET
from sklearn.preprocessing import MultiLabelBinarizer

def extract_labels_from_xml(xml_file_path):
    # Parse the XML file and get the root of the XML document
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Find the 'object' tags in the XML document
    objects = root.findall('object')

    # Extract the labels from all 'object' tags
    labels = [obj.find('name').text for obj in objects]

    return labels

# Specify the path to the 'annotations' directory
annotations_dir = r"/content/drive/MyDrive/Solar_PV/trainval/Annotations"
labels_train = []
for filename in os.listdir(train_path):
    if filename.endswith(".jpg"):
        # Extract the labels from the corresponding XML file
        xml_file_path = os.path.join(annotations_dir, filename.replace('.jpg', '.xml'))
        labels = extract_labels_from_xml(xml_file_path)

        labels_train.append(labels)

        path_train.append(os.path.join(train_path, filename))

# Apply MultiLabelBinarizer to the training labels
mlb = MultiLabelBinarizer()
labels_train_bin = mlb.fit_transform(labels_train)
# 0: Black core, 1: Finger
print(labels_train_bin)
print(mlb.classes_)

In [ ]:
print(len(mlb.classes_))

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Load and preprocess images
def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = cv2.imread(img_path)  # Load image using OpenCV
        img = cv2.resize(img, (desired_width, desired_height))  # Resize image
        img = img.astype(np.float32) / 255.0  # Normalize pixel values
        images.append(img)
    return np.array(images)

# Define desired width and height for resizing images
desired_width = 224
desired_height = 224

# Load and preprocess images
x_train = load_and_preprocess_images(path_train)

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, labels_train_bin, test_size=0.1, random_state=42)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from xml.etree import ElementTree as ET
def extract_boxes(filename):
    # load and parse the file
    tree = ET.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    names = list()
    for object in root.findall('.//object'):
        name = object.find('name').text
        names.append(name)
        box = object.find('bndbox')
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        coors = [xmin, ymin, xmax, ymax]
        boxes.append(coors)
    return boxes, names

In [ ]:
# visualize train image with multilabels

# Load image
from PIL import Image
im = np.array(Image.open(r'/kaggle/input/solar-cell-el-image/solar_cell_EL_image/PVELAD/EL2021/trainval/JPEGImages/img000005.jpg'))
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(im)

# Extract bounding boxes and object names from the corresponding XML file
boxes, names = extract_boxes(r'/kaggle/input/solar-cell-el-image/solar_cell_EL_image/PVELAD/EL2021/trainval/Annotations/img000005.xml')

# Create a Rectangle patch for each bounding box and add it to the plot
for box, name in zip(boxes, names):
    rect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.text(box[0], box[1], name, color='r')

# Show the plot with bounding boxes
plt.show()

In [ ]:
# from vit_keras import vit

# # Load the pre-trained Vision Transformer model
# vit_model = vit.vit_b16(
#     image_size=224,
#     activation='softmax',
#     pretrained=False,
#     include_top=True,
#     pretrained_top=False,
#     classes=12
# )

In [ ]:
# import tensorflow as tf
# from vit_keras import vit
# from tensorflow.keras.applications import MobileNet, EfficientNetB0, ResNet50

# # Load the pre-trained Vision Transformer model
# vit_model = vit.vit_b16(
#     image_size=224,
#     activation='softmax',
#     pretrained=False,
#     include_top=True,
#     pretrained_top=False,
#     classes=12
# )

# # Load the pre-trained MobileNet model
# mobilenet_model = MobileNet(
#     input_shape=(224, 224, 3),
#     alpha=1.0,
#     depth_multiplier=1,
#     dropout=0.001,
#     include_top=True,
#     weights=None,
#     classes=12
# )

# # Load the pre-trained EfficientNet model
# efficientnet_model = EfficientNetB0(
#     input_shape=(224, 224, 3),
#     include_top=True,
#     weights=None,
#     classes=12
# )

# # Load the pre-trained ResNet model
# resnet_model = ResNet50(
#     input_shape=(224, 224, 3),
#     include_top=True,
#     weights=None,
#     classes=12
# )

# # Print summaries of the models
# print("Vision Transformer Model Summary:")
# vit_model.summary()

# print("\nMobileNet Model Summary:")
# mobilenet_model.summary()

# print("\nEfficientNet Model Summary:")
# efficientnet_model.summary()

# print("\nResNet Model Summary:")
# resnet_model.summary()

# # Note: If your dataset is multi-label, ensure you are using appropriate loss functions and metrics when compiling the models.


Model Definitions


In [ ]:
import tensorflow as tf
from vit_keras import vit
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet, EfficientNetB0, ResNet50

# Load the pre-trained Vision Transformer model
vit_model = vit.vit_b16(
    image_size=224,
    activation='softmax',
    pretrained=False,
    include_top=True,
    pretrained_top=False,
    classes=12
)

# Load the pre-trained MobileNet model
mobilenet_base = MobileNet(
    input_shape=(224, 224, 3),
    include_top=False,
    weights=None
)
x = mobilenet_base.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(12, activation='softmax')(x)
mobilenet_model = Model(inputs=mobilenet_base.input, outputs=x)

# Load the pre-trained EfficientNet model
efficientnet_base = EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights=None
)
x = efficientnet_base.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(12, activation='softmax')(x)
efficientnet_model = Model(inputs=efficientnet_base.input, outputs=x)

# Load the pre-trained ResNet model
resnet_base = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights=None
)
x = resnet_base.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(12, activation='softmax')(x)
resnet_model = Model(inputs=resnet_base.input, outputs=x)

# Print summaries of the models
print("Vision Transformer Model Summary:")
vit_model.summary()

print("\nMobileNet Model Summary:")
mobilenet_model.summary()

print("\nEfficientNet Model Summary:")
efficientnet_model.summary()

print("\nResNet Model Summary:")
resnet_model.summary()


In [ ]:
# from tensorflow.keras.optimizers import Adam

# # Compile the model with binary crossentropy loss
# optimizer = Adam(learning_rate=0.0001)
# vit_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# from tensorflow.keras.optimizers import Adam

# # Compile the model with binary crossentropy loss
# optimizer = Adam(learning_rate=0.0001)
# mobilenet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# vit_model.summary()

In [ ]:
# import tensorflow as tf

# # Check if TensorFlow can access GPUs
# gpu_devices = tf.config.list_physical_devices('GPU')

# if gpu_devices:
#     print("Found GPU(s):")
#     for gpu in gpu_devices:
#         print(gpu)
# else:
#     print("No GPU(s) found. TensorFlow will use CPU.")


In [ ]:
# !nvidia-smi

In [ ]:
# with tf.device('/gpu:0'):
#     history = vit_model.fit(x_train, y_train, epochs=2, batch_size=8, validation_data=(x_val, y_val))


In [ ]:
# history = mobilenet_model.fit(x_train, y_train, epochs=50, batch_size=8, validation_data=(x_val, y_val))

In [ ]:
# # Assuming 'model' is your trained TensorFlow model
# vit_model.save('/content/drive/MyDrive/Solar_PV')


In [ ]:
# # Assuming 'model' is your trained TensorFlow model
# mobilenet_model.save_weights('/content/drive/MyDrive/Solar_PV/mnet_weights.h5')


In [ ]:
# plt.plot(history.history['accuracy'], label='train', color="red")
# plt.plot(history.history['val_accuracy'], label='validation', color="blue")
# plt.title('Model accuracy')
# plt.legend(loc='upper left')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.show()

# plt.plot(history.history['loss'], label='train', color="red")
# plt.plot(history.history['val_loss'], label='validation', color="blue")
# plt.title('Model loss')
# plt.legend(loc='upper left')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.show()

In [ ]:
# pred = vit_model.predict(x_val)

In [ ]:
# # Make predictions on the validation data
# y_pred = vit_model.predict(x_val)

# # Convert probabilities to binary predictions
# y_pred_binary = (y_pred > 0.5).astype(int)

# # Display results
# for i in range(len(y_val)):
#     print("Sample", i+1)
#     print("Actual Labels:", mlb.inverse_transform(np.array([y_val[i]])))
#     print("Predicted Labels:", mlb.inverse_transform(np.array([y_pred_binary[i]])))
#     print()


In [ ]:
# import matplotlib.pyplot as plt

# # Function to display image with predicted labels
# def display_image_with_labels(image, actual_labels, predicted_labels):
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image)
#     plt.axis('off')

#     # Add actual labels
#     actual_label_str = ', '.join(actual_labels[0])
#     plt.text(10, 20, f'Actual: {actual_label_str}', color='white', fontsize=10)

#     # Add predicted labels
#     predicted_label_str = ', '.join(predicted_labels[0])
#     plt.text(10, 40, f'Predicted: {predicted_label_str}', color='white', fontsize=10)

#     plt.show()

# # Display a sample image with its predicted labels
# sample_index = 0  # Index of the sample in the validation set
# sample_image = x_val[sample_index]
# actual_labels = mlb.inverse_transform(np.array([y_val[sample_index]]))
# predicted_labels = mlb.inverse_transform(np.array([y_pred_binary[sample_index]]))

# display_image_with_labels(sample_image, actual_labels, predicted_labels)


Model2


In [ ]:
# from tensorflow.keras.optimizers import Adam

# # Compile the model with binary crossentropy loss
# optimizer = Adam(learning_rate=0.0001)
# efficientnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# history = efficientnet_model.fit(x_train, y_train, epochs=50, batch_size=8, validation_data=(x_val, y_val))

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model with binary crossentropy loss
optimizer = Adam(learning_rate=0.0001)
resnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history =resnet_model.fit(x_train, y_train, epochs=10, batch_size=8, validation_data=(x_val, y_val))